In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, classification_report

2025-04-26 18:49:37.091039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745693377.267450      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745693377.318962      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="X5A7jopCvXj4ujuGEpUg")
project = rf.workspace("cleanskinaitest-k5soj").project("facial-emotion-recognition-nc1mc")
version = project.version(1)
dataset = version.download("folder")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:0000:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sigstore 3.6.1 requires rich~=13.0, but you have rich 14.0.0 which is incompatible.
datasets 3.5.0 requires fsspec[http]<=2024.

In [4]:
import os

path = '/kaggle/working/Facial-Emotion-Recognition-1'

for f in os.listdir(path):
    print(f)


README.dataset.txt
test
valid
README.roboflow.txt
train


In [5]:
import pandas as pd
import os

def create_dataframe_from_folder(folder_path):
    image_paths = []
    labels = []

    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for fname in os.listdir(label_path):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(label_path, fname))
                    labels.append(label)

    return pd.DataFrame({
        'filename': image_paths,
        'class': labels
    })

train_df = create_dataframe_from_folder('/kaggle/working/Facial-Emotion-Recognition-1/train')
valid_df = create_dataframe_from_folder('/kaggle/working/Facial-Emotion-Recognition-1/valid')
test_df = create_dataframe_from_folder('/kaggle/working/Facial-Emotion-Recognition-1/test')





In [6]:
train_df

,filename,class
0,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
1,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
2,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
3,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
4,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
...,...,...
15194,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
15195,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
15196,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
15197,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral


In [7]:
valid_df

,filename,class
0,/kaggle/working/Facial-Emotion-Recognition-1/v...,surprise
1,/kaggle/working/Facial-Emotion-Recognition-1/v...,surprise
2,/kaggle/working/Facial-Emotion-Recognition-1/v...,surprise
3,/kaggle/working/Facial-Emotion-Recognition-1/v...,surprise
4,/kaggle/working/Facial-Emotion-Recognition-1/v...,surprise
...,...,...
1428,/kaggle/working/Facial-Emotion-Recognition-1/v...,neutral
1429,/kaggle/working/Facial-Emotion-Recognition-1/v...,neutral
1430,/kaggle/working/Facial-Emotion-Recognition-1/v...,neutral
1431,/kaggle/working/Facial-Emotion-Recognition-1/v...,neutral


In [8]:
test_df

,filename,class
0,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
1,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
2,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
3,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
4,/kaggle/working/Facial-Emotion-Recognition-1/t...,surprise
...,...,...
717,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
718,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
719,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral
720,/kaggle/working/Facial-Emotion-Recognition-1/t...,neutral


In [18]:
gen = ImageDataGenerator()

train_gen = gen.flow_from_dataframe(train_df, x_col='filename', y_col='class',
                                    target_size=(224, 224), color_mode='rgb', class_mode='categorical', 
                                   batch_size=32)

valid_gen = gen.flow_from_dataframe(valid_df, x_col='filename', y_col='class',
                                    target_size=(224, 224), color_mode='rgb', class_mode='categorical', 
                                   batch_size=32)

test_gen = gen.flow_from_dataframe(test_df, x_col='filename', y_col='class',
                                    target_size=(224, 224), color_mode='rgb', class_mode='categorical', 
                                   batch_size=32,    shuffle=False  # ✅ very important for accurate predictions
)

Found 15199 validated image filenames belonging to 7 classes.
Found 1433 validated image filenames belonging to 7 classes.
Found 722 validated image filenames belonging to 7 classes.


In [10]:
base_model=tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling=None,
)

I0000 00:00:1745693398.244558      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
model = tf.keras.Sequential([
    base_model,
    Flatten(),

    tf.keras.layers.Dense(512 ,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(7, activation='softmax')  
])


model.compile(
    optimizer=Adamax(learning_rate=1e-5),
    loss='categorical_crossentropy',  
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      51,380,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,137,159 (286.63 MB)

 Trainable params: 75,082,247 (286.42 MB)

 Non-trainable params: 54,912 (214.50 KB)

In [12]:
hist = model.fit(train_gen, epochs=27, validation_data=valid_gen, batch_size=32)

Epoch 1/27


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
I0000 00:00:1745693448.324057      98 service.cc:148] XLA service 0x7ef3a402a5f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745693448.324906      98 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745693452.013578      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/475 ━━━━━━━━━━━━━━━━━━━━ 9:29:15 72s/step - accuracy: 0.0938 - loss: 3.5990

I0000 00:00:1745693473.589135      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


475/475 ━━━━━━━━━━━━━━━━━━━━ 181s 230ms/step - accuracy: 0.1608 - loss: 3.0713 - val_accuracy: 0.2470 - val_loss: 2.0048
Epoch 2/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 166ms/step - accuracy: 0.2167 - loss: 2.7305 - val_accuracy: 0.3398 - val_loss: 1.7318
Epoch 3/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 165ms/step - accuracy: 0.2672 - loss: 2.4426 - val_accuracy: 0.4215 - val_loss: 1.5494
Epoch 4/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 166ms/step - accuracy: 0.3155 - loss: 2.2567 - val_accuracy: 0.4976 - val_loss: 1.4060
Epoch 5/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 166ms/step - accuracy: 0.3569 - loss: 2.0568 - val_accuracy: 0.5520 - val_loss: 1.2527
Epoch 6/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 165ms/step - accuracy: 0.3973 - loss: 1.8896 - val_accuracy: 0.5932 - val_loss: 1.1412
Epoch 7/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 165ms/step - accuracy: 0.4353 - loss: 1.7552 - val_accuracy: 0.6211 - val_loss: 1.0545
Epoch 8/27
475/475 ━━━━━━━━━━━━━━━━━━━━ 79s 165ms/step - accuracy: 0.4758 - loss: 1.6118 - va

In [13]:
model.evaluate(test_gen)

23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.8770 - loss: 0.3755


[0.40482696890830994, 0.8753462433815002]

In [19]:
predictions = model.predict(test_gen)
predicted_classes = np.argmax(predictions, axis=1)


 2/23 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step 

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step


In [20]:
actual_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())


In [21]:
for i in range(len(actual_classes)):
    actual = class_labels[actual_classes[i]]
    predicted = class_labels[predicted_classes[i]]
    print(f"Image {i+1}: Actual: {actual} - Predicted: {predicted}")


Image 1: Actual: surprise - Predicted: surprise
Image 2: Actual: surprise - Predicted: surprise
Image 3: Actual: surprise - Predicted: surprise
Image 4: Actual: surprise - Predicted: surprise
Image 5: Actual: surprise - Predicted: surprise
Image 6: Actual: surprise - Predicted: surprise
Image 7: Actual: surprise - Predicted: surprise
Image 8: Actual: surprise - Predicted: surprise
Image 9: Actual: surprise - Predicted: surprise
Image 10: Actual: surprise - Predicted: surprise
Image 11: Actual: surprise - Predicted: surprise
Image 12: Actual: surprise - Predicted: surprise
Image 13: Actual: surprise - Predicted: surprise
Image 14: Actual: surprise - Predicted: surprise
Image 15: Actual: surprise - Predicted: surprise
Image 16: Actual: surprise - Predicted: surprise
Image 17: Actual: surprise - Predicted: surprise
Image 18: Actual: surprise - Predicted: surprise
Image 19: Actual: surprise - Predicted: surprise
Image 20: Actual: surprise - Predicted: surprise
Image 21: Actual: surprise - 

In [22]:
model.save('/kaggle/working/my_model.h5')
